# 使用 Gluon的线性回归 
 
 前一章使用ndarray和autograd 来实现线性回归，这一章我们仍然实现相同的模型，
 但是使用高层抽象包 gluon . 
 
 ## 创建数据集 
 
 我们生成同样的数据集 

In [1]:
from mxnet import ndarray as nd
from mxnet import autograd 
from mxnet import gluon

In [2]:
num_inputs=2 
num_examples=1000 
true_w=[2,-3.4]
true_b=4.2 

X = nd.random_normal(shape=(num_examples,num_inputs)) 
y=true_w[0]* X[:,0] + true_w[1]*X[:,1]+true_b 
y+=0.01 * nd.random_normal(shape=y.shape)

# 数据读取 
但这里使用data 模块来读取数据 。 

In [3]:
batch_size=10 
dataset=gluon.data.ArrayDataset(X,y) 
data_iter=gluon.data.DataLoader(dataset,batch_size,shuffle=True) 


读取和前面一致： 

In [4]:
for data,label in data_iter:
    print(data,label) 
    break 


[[ 8.1348163e-01 -5.6710786e-01]
 [ 2.5984043e-01  8.9015400e-01]
 [-2.4894042e-02  6.3270587e-01]
 [ 3.0029500e-01  7.3225945e-01]
 [ 1.0161712e+00  5.0922304e-01]
 [-3.4465104e-01 -4.1924685e-01]
 [-3.9304093e-01  4.5682105e-01]
 [ 2.4472828e-01  6.6647071e-01]
 [ 6.7431526e-04 -7.7242523e-01]
 [ 1.0347279e-01  1.3004491e-01]]
<NDArray 10x2 @cpu(0)> 
[7.776431  1.6905847 2.0036626 2.3162487 4.498421  4.9339895 1.867064
 2.4410765 6.8383746 3.9475276]
<NDArray 10 @cpu(0)>


# 定义模型 
当我们手写模型的时候，我们需要先声明模型参数，然后再使用它们来构建模型。但gluon提供大量
的提前定制好的层，使得我们只需要主要关注使用哪些层来构建模型。
例如线性模型就是使用对应的Dense层 

In [5]:
net=gluon.nn.Sequential() 

然后我们加入一个Dense层，它唯一必须要定义的参数就是输出节点的个数，在线性模型里面是1 

In [6]:
net.add(gluon.nn.Dense(1)) 

In [7]:
net

Sequential(
  (0): Dense(None -> 1, linear)
)

(注意这里我们并没有定义说这个层的输入节点是多少，在这之后真正给数据的时候系统会自动赋值。
我们之后会详细介绍这个特性是如何工作的。） 

# 初始化模型参数 
在使用前，net我们必须要初始化模型权重，这里我们使用默认随机初始化方法
(之后我们会介绍更多的初始化方法) 

In [8]:
net.initialize() 

# 损失函数 
gluon 提供了平方误差函数 

In [9]:
square_loss=gluon.loss.L2Loss() 

# 优化 
我们同样无需手动实现随机梯度下降，我们可以用创建一个Trainer的实例，并且将
模型参数传递给它就行 。 

In [10]:
trainer=gluon.Trainer(
    net.collect_params(),'sgd',{'learning_rate':0.01})

In [11]:
trainer

# 训练 
这里的训练跟前面没有太多区别，唯一的就是我们不再是调用SGD，而是trainer,step来攻心模型。

In [12]:
epochs=10
for e in range(epochs):
    total_loss=0 
    for data,label in data_iter: 
        with autograd.record():
            output=net(data) 
            loss=square_loss(output,label) 
        loss.backward()
        trainer.step(batch_size) 
        total_loss+=nd.sum(loss).asscalar()
    print("Epoch %d , average loss : %f "% (e,total_loss/num_examples)) 

Epoch 0 , average loss : 7.415231 
Epoch 1 , average loss : 0.970896 
Epoch 2 , average loss : 0.127767 
Epoch 3 , average loss : 0.016934 
Epoch 4 , average loss : 0.002291 
Epoch 5 , average loss : 0.000346 
Epoch 6 , average loss : 0.000088 
Epoch 7 , average loss : 0.000054 
Epoch 8 , average loss : 0.000049 
Epoch 9 , average loss : 0.000048 


In [13]:
dense=net[0]
true_w,dense.weight.data() 

([2, -3.4],
 
 [[ 1.9996634 -3.400032 ]]
 <NDArray 1x2 @cpu(0)>)

In [14]:
true_b,dense.bias.data()

(4.2,
 
 [4.2001705]
 <NDArray 1 @cpu(0)>)

# 结论
可以看到gluon可以帮助我们更快更干净地实现模型 

# 练习 

在训练的时候，为什么我们用了比前面要大10倍的学习率呢
（提示：可以尝试运行help(train.step)来查找答案) 
如何拿到weight的梯度呢?(提示：尝试help(dense.weight) ) 


In [15]:
help(dense.weight)

Help on Parameter in module mxnet.gluon.parameter object:

class Parameter(builtins.object)
 |  A Container holding parameters (weights) of Blocks.
 |  
 |  :py:class:`Parameter` holds a copy of the parameter on each :py:class:`Context` after
 |  it is initialized with ``Parameter.initialize(...)``. If :py:attr:`grad_req` is
 |  not ``'null'``, it will also hold a gradient array on each :py:class:`Context`::
 |  
 |      ctx = mx.gpu(0)
 |      x = mx.nd.zeros((16, 100), ctx=ctx)
 |      w = mx.gluon.Parameter('fc_weight', shape=(64, 100), init=mx.init.Xavier())
 |      b = mx.gluon.Parameter('fc_bias', shape=(64,), init=mx.init.Zero())
 |      w.initialize(ctx=ctx)
 |      b.initialize(ctx=ctx)
 |      out = mx.nd.FullyConnected(x, w.data(ctx), b.data(ctx), num_hidden=64)
 |  
 |  Parameters
 |  ----------
 |  name : str
 |      Name of this parameter.
 |  grad_req : {'write', 'add', 'null'}, default 'write'
 |      Specifies how to update gradient to grad arrays.
 |  
 |      - ``'wr

In [16]:
dense.weight?
